In [1]:
##  Parallel processing & retries with original prompt
import os
import io
import json
import pandas as pd
from dotenv import load_dotenv
import PyPDF2
from azure.storage.blob import ContainerClient
from openai import AzureOpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load environment variables
load_dotenv()

# Set up Azure Blob Storage connection
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
container_name = "earningcalltranscriptscontainer"  # Replace with your container name
container_client = ContainerClient.from_connection_string(
    conn_str=connection_string,
    container_name=container_name
)

# Function to extract text from a PDF file stream
def extract_text_from_pdf(file_stream):
    text = ""
    try:
        pdf_reader = PyPDF2.PdfReader(file_stream)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text

# Function to truncate text to fit within the token limit
def truncate_text(input_text, max_tokens):
    tokens = input_text.split()  # Naive tokenization by splitting on spaces
    if len(tokens) > max_tokens:
        truncated_text = ' '.join(tokens[:max_tokens])
        return truncated_text
    return input_text

# Function to interact with Azure OpenAI and generate JSON output
def get_json(file_stream):
    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version="2024-02-01"
    )

    # Extract text from PDF
    input_context = extract_text_from_pdf(file_stream)

    # Truncate the input context to avoid exceeding token limit
    max_input_tokens = 12000  # Set max input tokens (adjust as needed)
    input_context = truncate_text(input_context, max_input_tokens)

    # System and user instructions for the OpenAI model
    instructions = """
     As a Financial Analyst, you will leverage your expertise to generate tailored Financial Analysis 
     Reports that cater to the specific requirements of clients. 
    Key Objectives:

    Extract and Present: Provide detailed figures for each metric listed above, ensuring accurate and up-to-date data.
    Categorize Clearly: Organize the metrics under their respective categories as outlined.
    Ensure Completeness: Verify that all relevant metrics are included and presented with sufficient context for accurate interpretation.
    Highlight Capital Expenditure: Ensure that capital expenditure-related metrics are prominently detailed and clearly separated from other categories.
    """
    
    user_content = f"""
    To extract and organize financial metrics from the report, including those related to capital expenditure, profitability, liquidity, solvency, cash flow, and other key indicators. The metrics should be categorized and detailed according to their relevance to different aspects of financial analysis. 

    Operating Activities "Change in Working Capital",
            "Net Cash from Operating Activities"
    Investing Activities has "Acquisition of Fixed Assets & Intangibles",
            "Net Cash from Investing Activities"
    Financing Activities includes "Dividends Paid",
            "Cash from (Repayment of) Debt",
            "Net Cash from Financing Activities",
    Net Change includes  Net Change in Cash
    Metadata includes "Report Date", "Publish Date", "Source"
    Profitability Metrics like "EBITDA",
            "Gross Profit Margin",
            "Operating Margin",
            "Net Profit Margin",
            "Return on Equity",
            "Return on Assets",
            "Return On Invested Capital",
    Liquidity Metrics include Current Ratio
    Solvency Metrics like  "Total Debt", "Liabilities to Equity Ratio", "Debt Ratio",
    Cash Flow Metrics like "Free Cash Flow", "Free Cash Flow to Net Income", "Cash Return On Invested Capital",
    Other Important Metrics like "Piotroski F-Score", "Net Debt / EBITDA", "Dividend Payout Ratio
    Capital Expenditure 
    ```Document
    {input_context}
    ```
    """

    # JSON output structure request
    final_content = """
    Please provide the company name, year and quarter, capital expenditure value in billions of US dollars in JSON format.
    Use the following JSON structure:
    ```json
    {{
        "company": "",
        "year": "",
        "quarter": "",
        "capex": ""    
    }}
    ```
    Note: Only JSON Data is required, no other text is required.
    """

    # Steps to find the capital expenditure value
    how_content = """
    Please provide the steps how you found or calculated the capital expenditure value.
    """

    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_content}
    ]

    try:
        # Initial completion request
        response = client.chat.completions.create(
            model=os.getenv("DEPLOYMENT_NAME"),
            messages=messages,
        )
        response_content = response.choices[0].message.content
        messages.append({"role": "assistant", "content": response_content})
        messages.append({"role": "user", "content": final_content})
    except Exception as e:
        print(f"Error during OpenAI response generation: {e}")
        return None, None

    try:
        # Requesting JSON formatted data
        response_json = client.chat.completions.create(
            model=os.getenv("DEPLOYMENT_NAME"),
            messages=messages,
        )
        messages.append({"role": "assistant", "content": response_json.choices[0].message.content})
        messages.append({"role": "user", "content": how_content})

        # Requesting explanation of how the value was found
        how_response = client.chat.completions.create(
            model=os.getenv("DEPLOYMENT_NAME"),
            messages=messages
        )
    except Exception as e:
        print(f"Error during JSON extraction or explanation generation: {e}")
        return None, None

    return response_json.choices[0].message.content, how_response.choices[0].message.content

# Function to process a single blob with retry logic
def process_blob(blob, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            stream = io.BytesIO()
            container_client.download_blob(blob).readinto(stream)
            op, how = get_json(stream)
            if op and how:
                op = json.loads(op[op.index("{"):len(op)-op[::-1].index("}")])
                if op.get("capex"):  # Check if capex is not empty
                    return op, how
                else:
                    print(f"Capex is empty. Retrying for blob {blob.name}...")
            attempt += 1
            time.sleep(5)  # Wait before retrying
        except Exception as e:
            print(f"Error processing blob {blob.name}: {e}")
            return None, None
    print(f"Max retries reached for blob {blob.name}.")
    return None, None

# Main function to process blobs in parallel
def main():
    final_json = []
    blob_list = list(container_client.list_blobs())[:40]  # Limit to the first 20 blobs

    # Using ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_blob = {executor.submit(process_blob, blob): blob for blob in blob_list}
        # Iterate over completed futures
        for future in as_completed(future_to_blob):
            blob = future_to_blob[future]
            try:
                result = future.result()
                if result:
                    op, how = result
                    if op:  # Ensure op is not None
                        print(op, "\n\n", how)
                        final_json.append(op)
            except Exception as e:
                print(f"Error processing result for blob {blob.name}: {e}")

    # Ensure final_json contains valid data
    final_json = [item for item in final_json if isinstance(item, dict)]
    # Optional: Save final JSON to a file
    output_file = "financial_analysis_results_1201.json"
    with open(output_file, "w") as outfile:
        json.dump(final_json, outfile, indent=4)

    # Convert final_json to DataFrame and save to CSV
    if final_json:  # Check if final_json is not empty
        df = pd.DataFrame(final_json)
        output_file = "financial_analysis_results10.csv"
        df.to_csv(output_file, index=False)
        print(f"Processing completed. Results saved to {output_file}.")
    else:
        print("No valid data to save to CSV.")

if __name__ == "__main__":
    main()


{'company': 'Alphabet Inc.', 'year': '2021', 'quarter': 'Q2', 'capex': '5.5'} 

 Apologies for the confusion. The provided report does not directly specify the capital expenditure value. In order to calculate the capital expenditure, we would need additional information, such as the breakdown of the Purchases of Property and Equipment for the specific quarter. Unfortunately, the provided report does not include the detailed breakdown of capital expenditures. Without further information, it is not possible to calculate the exact capital expenditure value.
{'company': 'Alphabet Inc.', 'year': '2021', 'quarter': 'Q3', 'capex': '6.8'} 

 The capital expenditure value can be found by reviewing the "Consolidated Statements of Cash Flows" section of the financial report. Specifically, the line item "Purchases of property and equipment" represents the capital expenditure made by the company during the specified quarter.

In this case, the capital expenditure value of $6.8 billion for Alphabet 

In [13]:
## Rearranged input capex with quarter_year with company name 
import pandas as pd

# Read the original CSV file
input_file = "financial_analysis_results10.csv"
df = pd.read_csv(input_file)

# Convert 'quarter' and 'year' columns into a single 'Quarter-Year' column with an underscore
df['Quarter-Year'] = df['quarter'] + "_" + df['year'].astype(str)

# Pivot the DataFrame
pivot_df = df.pivot_table(index='company', columns='Quarter-Year', values='capex', aggfunc='first')

# Reset the index to make 'company' a column instead of an index
pivot_df.reset_index(inplace=True)

# Define a function to sort columns in the desired order
def sort_columns(df):
    # Extract the current columns
    columns = df.columns.tolist()
    
    # Extract the company column and the Quarter-Year columns
    company_col = columns[0]
    quarter_cols = columns[1:]
    
    # Generate sorted columns list: Start with the most recent quarters
    sorted_quarters = sorted(quarter_cols, key=lambda x: (x.split('_')[1], x.split('_')[0]), reverse=True)
    
    # Combine sorted columns with the company column
    sorted_columns = [company_col] + sorted_quarters
    return sorted_columns

# Reorder the columns
sorted_columns = sort_columns(pivot_df)
pivot_df = pivot_df[sorted_columns]

# Save the rearranged DataFrame to a new CSV file
output_file = "rearranged_financial_analysis_results1.csv"
pivot_df.to_csv(output_file, index=False)

print(f"Rearranged data saved to {output_file}.")


Rearranged data saved to rearranged_financial_analysis_results1.csv.


In [15]:
import pandas as pd
import numpy as np

# Load the existing financial data
input_file = 'rearranged_financial_analysis_results1.csv'
df = pd.read_csv(input_file)

# Define quarters and years in the desired order
quarters = ['Q4', 'Q3', 'Q2', 'Q1']
years = list(range(2024, 2013, -1))  # From 2024 to 2014
quarters_years = [f"{q}_{y}" for y in years for q in quarters]

# Create the template DataFrame with these columns
template_df = pd.DataFrame(columns=['company'] + quarters_years)

# Standardize company names
df['company'] = df['company'].replace({
    'Amazon': 'Amazon.com',
    'Amazon.com, Inc.': 'Amazon.com',
    'Alphabet Inc':'Alphabet Inc.'
})

# Create a new DataFrame to hold aggregated data
aggregated_data = df.groupby('company').sum(numeric_only=True).reset_index()

# Initialize the template DataFrame
template_data = {'company': list(aggregated_data['company'])}
for quarter in quarters_years:
    template_data[quarter] = [np.nan] * len(template_data['company'])

template_df = pd.DataFrame(template_data)

# Merge the aggregated data with the template DataFrame
# First, ensure the column names in aggregated_data match the template_df
matching_columns = [col for col in aggregated_data.columns if col in template_df.columns]
merged_df = pd.merge(template_df, aggregated_data, on='company', how='left', suffixes=('', '_agg'))

# Map the values from aggregated_data to template_df
for column in quarters_years:
    if column in matching_columns:
        merged_df[column] = merged_df[f"{column}_agg"]

# Drop the extra columns used during the merge
final_df = merged_df.drop(columns=[col for col in merged_df.columns if col.endswith('_agg')])

# Fill missing values with NaN
for column in final_df.columns[1:]:
    final_df[column] = final_df[column].fillna(np.nan)
# Replace 0 with NaN
final_df.replace(0, np.nan, inplace=True)
# Save the result to a new CSV file if needed
final_df.to_csv('combined_financial_analysis_results.csv', index=False)

#print(final_df)


In [17]:
# compare the Capex with previous quarter value and decide either increase, decrease or data unavailable, unchanged

# Read the CSV file
input_file = "combined_financial_analysis_results.csv"
df = pd.read_csv(input_file)

# Define a function to compare capex values between consecutive columns
def compare_columns(row):
    comparisons = []
    for i in range(1, len(row) - 1):  # Skip the 'company' column
        current_value = row[i]
        next_value = row[i + 1]
        
        if pd.isna(current_value) or pd.isna(next_value):
            comparisons.append("Data not available")
        else:
            try:
                current_value = float(current_value)
                next_value = float(next_value)
                if next_value < current_value:
                    comparisons.append("increase")
                elif next_value > current_value:
                    comparisons.append("decrease")
                else:
                    comparisons.append("unchanged")
            except ValueError:
                comparisons.append("Data not available")
    
    return comparisons

# Apply the comparison function to each row
comparison_results = df.apply(lambda row: compare_columns(row), axis=1)

# Generate column names for comparison results
quarter_columns = df.columns[1:]  # Exclude 'company'
comparison_columns = [f"Compare_{quarter_columns[i]}_{quarter_columns[i+1]}" for i in range(len(quarter_columns) - 1)]

# Create DataFrame for comparison results
comparison_df = pd.DataFrame(comparison_results.tolist(), columns=comparison_columns)

# Combine the company column with the comparison results
final_comparison_df = pd.concat([df[['company']], comparison_df], axis=1)

# Save the comparison results to a new CSV file
output_file = "capex_comparison_results.csv"
final_comparison_df.to_csv(output_file, index=False)

print(f"Capex comparison results saved to {output_file}.")


Capex comparison results saved to capex_comparison_results.csv.


/tmp/ipykernel_75725/439761825.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_value = row[i]
/tmp/ipykernel_75725/439761825.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_value = row[i + 1]
